In [ ]:
# đọc file
filename='train.txt'
lines=[]
count=0
#Max=-1
Max=10000
with open(filename,'r') as f:
    for s in f:
        count+=1
        if count>Max and Max!=-1:
            break
        lines.append(s.strip())
print(len(lines))
#print(lines[:5])          

10000


In [ ]:
for i in range(5):
  print(lines[i])  

Here are two reasons companies fail: they only do more of the same, or they only do what's new
To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation
Both are necessary, but it can be too much of a good thing
Consider Facit
I'm actually old enough to remember them


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# tokenize sentences 
import nltk
sentences=[]
all_tokens_count=0
for line in lines:
    tokens = nltk.word_tokenize(line.lower())
    all_tokens_count+=len(tokens)
    #sentences.append(tokens)
    sentences.append(['<s>']+tokens+['</s>'])
print('all_tokens_count=',all_tokens_count)
print(len(sentences))
print(sentences[:5])

all_tokens_count= 185921
10000
[['<s>', 'here', 'are', 'two', 'reasons', 'companies', 'fail', ':', 'they', 'only', 'do', 'more', 'of', 'the', 'same', ',', 'or', 'they', 'only', 'do', 'what', "'s", 'new', '</s>'], ['<s>', 'to', 'me', 'the', 'real', ',', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', ':', 'exploration', 'and', 'exploitation', '</s>'], ['<s>', 'both', 'are', 'necessary', ',', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing', '</s>'], ['<s>', 'consider', 'facit', '</s>'], ['<s>', 'i', "'m", 'actually', 'old', 'enough', 'to', 'remember', 'them', '</s>']]


In [ ]:
# counting 1-gram 
from collections import Counter
counter_unigram=Counter()
for sent in sentences:
    counter_unigram.update(sent)
V=len(counter_unigram)
print('V=',V)
n=0
for gram in counter_unigram:
    n+=counter_unigram[gram]
n=n-counter_unigram['<s>']-counter_unigram['</s>']
print('n=',n)
print(counter_unigram['the'])
print(counter_unigram['he'])

V= 12839
n= 185921
8142
418


In [ ]:
from nltk import ngrams
input_sent='the human body with knew abilities is know longer a question'
gram2=ngrams(['<s>']+input_sent.split(),2)
for gram in gram2:
    print(gram)

('<s>', 'the')
('the', 'human')
('human', 'body')
('body', 'with')
('with', 'knew')
('knew', 'abilities')
('abilities', 'is')
('is', 'know')
('know', 'longer')
('longer', 'a')
('a', 'question')


In [ ]:
from nltk import ngrams

bi_grams=[]
for sent in sentences:
    gram2=ngrams(sent,2)
    bi_grams.extend(gram2)
print(len(bi_grams))

for i in range(3):
    print(bi_grams[i])

counter_bigram = Counter(bi_grams)
print('bigram number =',len(counter_bigram))
print(counter_bigram[('here','are')])

195921
('<s>', 'here')
('here', 'are')
('are', 'two')
bigram number = 80118
10


In [ ]:
tri_grams=[]
for sent in sentences:
    gram3=ngrams(sent,3)
    tri_grams.extend(gram3)
print(len(tri_grams))

for i in range(3):
    print(tri_grams[i])

counter_trigram = Counter(tri_grams)
print('trigram number =',len(counter_trigram))
print(counter_trigram[('here','are','two')])

185921
('<s>', 'here', 'are')
('here', 'are', 'two')
('are', 'two', 'reasons')
trigram number = 142548
2


In [ ]:
# tính theo Laplace 
anpha = 0.001
def uni_prob(word):
    return max(1,counter_unigram[word])/all_tokens_count
def bi_prob(word1,word2):
    return (counter_bigram[(word1,word2)]+anpha)/(counter_unigram[word1]+V*anpha)
    
def tri_prob(word1,word2,word3):
    return (counter_trigram[(word1,word2,word3)]+anpha)/(counter_bigram[(word1,word2)]+anpha*V)
    

In [ ]:
# tính xác suất của một câu, normalize theo 1 từ 
def probLM(sent,n):
    if n>3 or n<1: # không xét trường hợp này 
        return 0
    tokens=nltk.word_tokenize(sent.lower())
    tokens = ['<s>']+tokens
    prob=1
    for i in range(1,len(tokens)):
        if n==1:
            prob*=uni_prob(tokens[i])
        elif n==2:
            prob*=bi_prob(tokens[i-1],tokens[i])
        elif n==3:
            if i>=2:
                prob*=tri_prob(tokens[i-2],tokens[i-1],tokens[i])
            else:
                prob*=bi_prob(tokens[i-1],tokens[i])
    k=len(tokens)-1
    return prob**(1/k)

In [ ]:
sent='the human body with new abilities is no longer a question'
#sent='the human body with new from abilities is no longer a question'
#sent='A few years back from'
print('n=1')
pr=probLM(sent,1)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=2')
pr=probLM(sent,2)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=3')
pr=probLM(sent,3)
print('prob=',pr)
print('perplexity=',1/pr)

n=1
prob= 0.0013984311264167914
perplexity= 715.0870579964178
n=2
prob= 0.015576696673651823
perplexity= 64.1984639587617
n=3
prob= 0.027047801941636494
perplexity= 36.97158098679483


In [ ]:
# kiểm tra xem 2 xâu có xác suất hơn nhau thế nào, ví dụ cho bài toán speech to text
sent1='the human body with new abilities is no longer a question'
sent2='the human body with knew abilities is know longer a question'
pr=probLM(sent1,3)
print('prob=',pr)
print('perplexity=',1/pr)

pr=probLM(sent2,3)
print('prob=',pr)
print('perplexity=',1/pr)

prob= 0.027047801941636494
perplexity= 36.97158098679483
prob= 0.0007085628998843219
perplexity= 1411.3073097155627


In [ ]:
sent1='the human body with new abilities is no longer a question'
sent2='the human body with knew abilities is know longer a question'

def stp_back_off(sent):
  # start with triagram and
  tokens=nltk.word_tokenize(sent.lower())
  tokens = ['<s>']+tokens
  prob=1
  for i in range(3,len(tokens)):
    if tri_prob(tokens[i-2],tokens[i-1],tokens[i]) == 0:
      if bi_prob(tokens[i-1],tokens[i]) == 0:
        prob*= 0.4*0.4*uni_prob(tokens[i])
      else:
        prob*= 0.4*bi_prob(tokens[i-1],tokens[i])
    else:
      prob*=tri_prob(tokens[i-2],tokens[i-1],tokens[i])
  k=len(tokens)-1
  return prob**(1/k)

# stupid back off implementation
print("stupid back off")
pr=stp_back_off(sent1)
print('prob=',pr)
print('perplexity sent1=',1/pr)

pr=stp_back_off(sent2)
print('prob=',pr)
print('perplexity sent2=',1/pr)
# laplace 
print("\nlaplace")
pr=probLM(sent1,3)
print('prob=',pr)
print('perplexity sent1=',1/pr)

pr=probLM(sent2,3)
print('prob=',pr)
print('perplexity sent2=',1/pr)

stupid back off
prob= 0.1174173359939537
perplexity sent1= 8.516629946803546
prob= 0.003075945626490854
perplexity sent2= 325.10327600973716

laplace
prob= 0.027047801941636494
perplexity sent1= 36.97158098679483
prob= 0.0007085628998843219
perplexity sent2= 1411.3073097155627
